In [1]:
import pandas as pds
import numpy as np
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals()) # define pysqldf function for queries

## Load MIxS 5 enviromental package data

In [2]:
df = pds.read_excel("data/mixs_v5.xlsx", sheet_name="environmental_packages")

In [3]:
df.head()

,Environmental package,Structured comment name,Package item,Definition,Expected value,Value syntax,Example,Requirement,Preferred unit,Occurrence,Position,MIXS ID
0,air,alt,altitude,Altitude is a term used to identify heights of...,measurement value,{float} {unit},100 meter,M,meter,1,0,MIXS:0000094
1,air,elev,elevation,Elevation of the sampling site is its height a...,measurement value,{float} {unit},100 meter,C,meter,1,0,MIXS:0000095
2,air,barometric_press,barometric pressure,Force per unit area exerted against a surface ...,measurement value,{float} {unit},5 millibar,X,millibar,1,1,MIXS:0000096
3,air,carb_dioxide,carbon dioxide,Carbon dioxide (gas) amount or concentration a...,measurement value,{float} {unit},410 parts per million,X,"micromole per liter, parts per million",1,1,MIXS:0000097
4,air,carb_monoxide,carbon monoxide,Carbon monoxide (gas) amount or concentration ...,measurement value,{float} {unit},0.1 parts per million,X,"micromole per liter, parts per million",1,1,MIXS:0000098


### Find distinct terms and number of occurrences for each term 
Just doing this for **curiousity** in order to get a feel for the data

In [4]:
q = """
select
    [Package item], Definition, count(*) as count
from
    df
group by
    [Package item], Definition
order by
    count(*) desc
"""
pysqldf(q)

,Package item,Definition,count
0,miscellaneous parameter,Any other measurement performed or parameter c...,16
1,organism count,Total cell count of any organism (or group of ...,16
2,sample volume or weight for DNA extraction,"Volume (ml), weight (g) of processed sample, o...",16
3,oxygenation status of sample,Oxygenation status of sample,15
4,sample storage duration,Duration for which the sample was stored,15
...,...,...,...
505,window open frequency,The number of times windows are opened per week,1
506,window signs of water/mold,Signs of the presence of mold or mildew on the...,1
507,window status,Defines whether the windows were open or close...,1
508,window type,The type of windows,1


Based on above queries:  
Total number of distinct terms: **509**  
Total number of distinct terms in > 1 package: **147**  

## Create data frame of distinct field name / package name pairings

In [6]:
q = """
select distinct
    [Package item] as field, [Definition] as definition, [Environmental package] as package, count(*) as field_count
from
    df
group by
    [Package item], [Definition], [Environmental package]
order by
    [Package item], [Environmental package], [Definition]
"""
fieldsdf = pysqldf(q)

In [7]:
fieldsdf.head() # peek at data

,field,definition,package,field_count
0,API gravity,API gravity is a measure of how heavy or light...,hydrocarbon resources-cores,1
1,API gravity,API gravity is a measure of how heavy or light...,hydrocarbon resources-fluids/swabs,1
2,HRT,Whether subject had hormone replacement therap...,human-vaginal,1
3,IHMC ethnicity,Ethnicity of the subject,human-associated,1
4,IHMC ethnicity,Ethnicity of the subject,human-gut,1


### Pivot the fields data frame so that the package values are columns with the field_count as the column values

In [8]:
pivotdf = fieldsdf.pivot_table(index=['field', 'definition'], columns=['package'], values=['field_count'], aggfunc=np.sum, fill_value=0)


In [9]:
pds.set_option('display.max_columns', 999) # display all columns
pivotdf.head() # peek at data

field_count  \
package                                                                          air   
field                 definition                                                       
API gravity           API gravity is a measure of how heavy or light ...           0   
HRT                   Whether subject had hormone replacement theraph...           0   
IHMC ethnicity        Ethnicity of the subject                                     0   
IHMC medication code  Can include multiple medication codes                        0   
absolute air humidity Actual mass of water vapor - mh20 - present in ...           0   

                                                                                            \
package                                                                  built environment   
field                 definition                                                             
API gravity           API gravity is a measure of how heavy or light ...                 0   
HRT                   Whether subject had hormone replacement theraph...                 0   
IHMC ethnicity        Ethnicity of the subject                                           0   
IHMC medication code  Can include multiple medication codes                              0   
absolute air humidity Actual mass of water vapor - mh20 - present in ...                 1   

                                                                                          \
package                                                                  host-associated   
field                 definition                                                           
API gravity           API gravity is a measure of how heavy or light ...               0   
HRT                   Whether subject had hormone replacement theraph...               0   
IHMC ethnicity        Ethnicity of the subject                                         0   
IHMC medication code  Can include multiple medication codes                            0   
absolute air humidity Actual mass of water vapor - mh20 - present in ...               0   

                                                                                           \
package                                                                  human-associated   
field                 definition                                                            
API gravity           API gravity is a measure of how heavy or light ...                0   
HRT                   Whether subject had hormone replacement theraph...                0   
IHMC ethnicity        Ethnicity of the subject                                          1   
IHMC medication code  Can include multiple medication codes                             1   
absolute air humidity Actual mass of water vapor - mh20 - present in ...                0   

                                                                                    \
package                                                                  human-gut   
field                 definition                                                     
API gravity           API gravity is a measure of how heavy or light ...         0   
HRT                   Whether subject had hormone replacement theraph...         0   
IHMC ethnicity        Ethnicity of the subject                                   1   
IHMC medication code  Can include multiple medication codes                      1   
absolute air humidity Actual mass of water vapor - mh20 - present in ...         0   

                                                                                     \
package                                                                  human-oral   
field                 definition                                                      
API gravity           API gravity is a measure of how heavy or light ...          0   
HRT                   Whether subject had hormone replacement theraph...          0   
IHMC ethn

### Perform some data clean up
- drop the coloum hierarchical index (if scroll to the right, you will see the name 'field_count' above 'water')
- add a column to sum up the number of packages a term occurrs in (this may allow for easy filtering)
- reset the index

In [10]:
pivotdf.columns = pivotdf.columns.droplevel()

In [11]:
pivotdf['total'] = pivotdf.sum(axis=1)

In [12]:
pivotdf.columns.name = "" # this removes the 'package' label from the left side of table

In [13]:
pivotdf.reset_index(inplace=True) # makes the field and definitions to be column names

In [14]:
pivotdf.head() # peek at data

,field,definition,air,built environment,host-associated,human-associated,human-gut,human-oral,human-skin,human-vaginal,hydrocarbon resources-cores,hydrocarbon resources-fluids/swabs,microbial mat/biofilm,miscellaneous natural or artificial environment,plant-associated,sediment,soil,wastewater/sludge,water,total
0,API gravity,API gravity is a measure of how heavy or light...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2
1,HRT,Whether subject had hormone replacement therap...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,IHMC ethnicity,Ethnicity of the subject,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5
3,IHMC medication code,Can include multiple medication codes,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5
4,absolute air humidity,Actual mass of water vapor - mh20 - present in...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Attach MIXS IDs to package-term counts
- create dataframe containing a list of MIXS IDs for each term and definition
- join MIXS IDs to pivot table

In [15]:
q = """
select distinct
    [Package item] as field, [Definition] as definition, group_concat([MIXS ID], ", ") as mixs_ids
from
    df
group by
    [Package item], [Definition]
order by
    [Package item], [Definition]
"""
mixs_id_df = pysqldf(q)

In [16]:
mixs_id_df.head() # peek at data

,field,definition,mixs_ids
0,API gravity,API gravity is a measure of how heavy or light...,"MIXS:0000444, MIXS:0000157"
1,HRT,Whether subject had hormone replacement therap...,MIXS:0000969
2,IHMC ethnicity,Ethnicity of the subject,"MIXS:0000895, MIXS:0000916, MIXS:0000935, MIXS..."
3,IHMC medication code,Can include multiple medication codes,"MIXS:0000884, MIXS:0000908, MIXS:0000928, MIXS..."
4,absolute air humidity,Actual mass of water vapor - mh20 - present in...,MIXS:0000122


In [19]:
q = """
select distinct
    mixs_id_df.[field], mixs_id_df.[definition], pivotdf.[air], pivotdf.[built environment], pivotdf.[host-associated], pivotdf.[human-associated], pivotdf.[human-gut], pivotdf.[human-oral], pivotdf.[human-skin], pivotdf.[human-vaginal], pivotdf.[hydrocarbon resources-cores], pivotdf.[hydrocarbon resources-fluids/swabs], pivotdf.[microbial mat/biofilm], pivotdf.[miscellaneous natural or artificial environment], pivotdf.[plant-associated], pivotdf.[sediment], pivotdf.[soil], pivotdf.[wastewater/sludge], pivotdf.[water], pivotdf.[total], mixs_id_df.[mixs_ids]
from
    mixs_id_df
left join
    pivotdf
on
    mixs_id_df.[field] = pivotdf.[field]
and mixs_id_df.[definition] = pivotdf.[definition]
"""
term_countdf = pysqldf(q).fillna(0)

In [20]:
term_countdf.head() # peek at data

,field,definition,air,built environment,host-associated,human-associated,human-gut,human-oral,human-skin,human-vaginal,hydrocarbon resources-cores,hydrocarbon resources-fluids/swabs,microbial mat/biofilm,miscellaneous natural or artificial environment,plant-associated,sediment,soil,wastewater/sludge,water,total,mixs_ids
0,API gravity,API gravity is a measure of how heavy or light...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,"MIXS:0000444, MIXS:0000157"
1,HRT,Whether subject had hormone replacement therap...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,MIXS:0000969
2,IHMC ethnicity,Ethnicity of the subject,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5,"MIXS:0000895, MIXS:0000916, MIXS:0000935, MIXS..."
3,IHMC medication code,Can include multiple medication codes,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5,"MIXS:0000884, MIXS:0000908, MIXS:0000928, MIXS..."
4,absolute air humidity,Actual mass of water vapor - mh20 - present in...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MIXS:0000122


### Create spreadsheet of terms that exist in multiple packages

In [21]:
multi_termdf = term_countdf[term_countdf.total > 1]

In [22]:
multi_termdf.head()

,field,definition,air,built environment,host-associated,human-associated,human-gut,human-oral,human-skin,human-vaginal,hydrocarbon resources-cores,hydrocarbon resources-fluids/swabs,microbial mat/biofilm,miscellaneous natural or artificial environment,plant-associated,sediment,soil,wastewater/sludge,water,total,mixs_ids
0,API gravity,API gravity is a measure of how heavy or light...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,"MIXS:0000444, MIXS:0000157"
2,IHMC ethnicity,Ethnicity of the subject,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5,"MIXS:0000895, MIXS:0000916, MIXS:0000935, MIXS..."
3,IHMC medication code,Can include multiple medication codes,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,5,"MIXS:0000884, MIXS:0000908, MIXS:0000928, MIXS..."
5,additional info,Information that doesn't fit anywhere else. Ca...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,"MIXS:0000300, MIXS:0000309"
11,alkalinity,"Alkalinity, the ability of a solution to neutr...",0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,1,1,7,"MIXS:0000421, MIXS:0000468, MIXS:0000487, MIXS..."


In [23]:
len(multi_termdf)

147

### Save spreadsheets

In [24]:
term_countdf.to_excel("output/mixs-package-term.xlsx", engine='xlsxwriter')

In [25]:
multi_termdf.to_excel("output/multi-package-mixs-terms-only.xlsx", engine='xlsxwriter')